# **Taller 06: Bases de datos Vectoriales**

Nombres: Rossy Armendariz, Alejandro Chavez, Wilmer Rivas.

## Parte 1: Recuperación con TF-IDF

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Leer el archivo CSV que contiene los datos de las películas
data = pd.read_csv('./data/wiki_movie_plots_deduped.csv')

# Filtrar solo las columnas "Title" y "Plot" necesarias para el análisis
selected_data = data[['Title', 'Plot']]

# Visualizar las primeras filas del DataFrame resultante
print(selected_data.head())

In [ ]:
# Configuración del vectorizador TF-IDF para procesar el texto
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Generación de la matriz TF-IDF basada en la columna 'Plot'
tfidf_matrix = tfidf_vectorizer.fit_transform(selected_data['Plot'])

# Definición de la función para ejecutar búsquedas basadas en TF-IDF
def tfidf_search(query_text, results_count=5):
  
    # Transformar la consulta en vector TF-IDF
    query_vector = tfidf_vectorizer.transform([query_text])
    
    # Calcular la similitud entre la consulta y los documentos
    similarity_scores = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Seleccionar los índices de los documentos más relevantes
    best_match_indices = similarity_scores.argsort()[-results_count:][::-1]
    
    # Crear un DataFrame con los resultados
    top_results = selected_data.iloc[best_match_indices][['Title', 'Plot']].copy()
    top_results['Score'] = similarity_scores[best_match_indices]
    
    return top_results

# Ejemplo de consulta
search_query = "A young boy discovers magic"
search_results = tfidf_search(search_query)

# Mostrar los resultados de la búsqueda
print(search_results)

## Parte 2: Recuperación con BM25

%pip install elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import pandas as pd

# Establecer conexión con el servidor Elasticsearch
elastic_client = Elasticsearch("http://localhost:9200")  # Incluyendo el esquema http

# Nombre del índice a manejar
index_name = "movie_plots"

# Verificar si el índice ya existe, y eliminarlo si es necesario
if elastic_client.indices.exists(index=index_name):
    elastic_client.indices.delete(index=index_name)
    print(f"Índice '{index_name}' eliminado correctamente.")

# Crear un índice nuevo en Elasticsearch
elastic_client.indices.create(index=index_name)
print(f"Índice '{index_name}' creado con éxito.")


In [ ]:
# Función para cargar los datos en el índice existente
def cargar_datos():
    for fila, datos in selected_data.iterrows():  # Usamos el DataFrame filtrado
        yield {
            "_index": index_name,
            "_source": {
                "Indice": int(fila),  # Almacena el índice original convertido a un entero
                "Titulo": datos['Title'],
                "Sinopsis": datos['Plot']
            }
        }

# Enviar los documentos al índice
bulk(elastic_client, cargar_datos())
print("Proceso de indexación finalizado.")

In [ ]:
# Función para realizar consultas con BM25
def search_bm25(query, top_k=5):
    # Construir el cuerpo de la consulta
    query_body = {
        "query": {
            "match": {
                "Sinopsis": query  # Cambiar "Plot" por "Sinopsis" si así se indexaron los datos
            }
        },
        "size": top_k  # Limitar el número de resultados
    }

    # Ejecutar la consulta en Elasticsearch
    response = elastic_client.search(index=index_name, body=query_body)

    # Crear una lista para almacenar los resultados
    results = []
    for hit in response['hits']['hits']:
        # Asegurarse de manejar el caso en que 'Indice' no exista
        results.append({
            "Index": hit['_source'].get('Indice', None),  # Usar .get() para evitar errores
            "Title": hit['_source'].get('Titulo', "Unknown Title"),
            "Plot": hit['_source'].get('Sinopsis', "No Plot Available"),
            "Score": hit['_score']  # Puntuación de relevancia
        })

    # Convertir la lista en un DataFrame
    # Usar .set_index solo si "Index" tiene valores válidos
    df_results = pd.DataFrame(results)
    if 'Index' in df_results.columns and df_results['Index'].notnull().all():
        df_results = df_results.set_index("Index")
    
    return df_results

# Ejemplo de ejecución
query = "A young boy discovers magic"  # Texto de búsqueda
results = search_bm25(query)  # Recuperar los documentos relevantes
print(results)  # Mostrar los resultados


## Parte 3: Recuperación con FAISS

## Parte 4: Recuperación con ChromaDB

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


True
12.4
NVIDIA GeForce RTX 3050 Ti Laptop GPU


Configuración de ChromaDB y embeddings

In [3]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
import torch

# Verificar si la GPU está disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando el dispositivo: {device}")

# Cargar el archivo CSV de películas
movies_df = pd.read_csv("./wiki_movie_plots_deduped.csv")

# Filtrar campos necesarios
movies_df = movies_df[movies_df['Plot'].notna()]  # Eliminar filas con Plot vacío
documents = movies_df['Plot'].tolist()  # Lista de las tramas
titles = movies_df['Title'].tolist()   # Lista de los títulos

# Configurar cliente Chroma
client = chromadb.Client()

# Crear la función de embeddings con GPU
embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2",  # Modelo eficiente y rápido
    device=device  # Especificar el dispositivo (GPU o CPU)
)

# Crear colección con la función de embeddings configurada
collection = client.get_or_create_collection(
    name="movies_collection",
    embedding_function=embedding_function
)

# Generar IDs únicos para cada documento
ids = [f"movie_{i}" for i in range(len(movies_df))]

# Dividir los datos en lotes
batch_size = 5000  # Tamaño del lote (ajustar según sea necesario)
for i in range(0, len(documents), batch_size):
    batch_documents = documents[i:i+batch_size]
    batch_titles = titles[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]
    
    # Agregar el lote a la colección
    collection.add(
        documents=batch_documents,  # Tramas del lote
        metadatas=[{"title": title} for title in batch_titles],  # Metadata del lote
        ids=batch_ids  # IDs únicos del lote
    )
    print(f"Lote {i // batch_size + 1} procesado exitosamente.")

print(f"Se han agregado {len(documents)} documentos y generado sus embeddings.")


Usando el dispositivo: cuda
Lote 1 procesado exitosamente.
Lote 2 procesado exitosamente.
Lote 3 procesado exitosamente.
Lote 4 procesado exitosamente.
Lote 5 procesado exitosamente.
Lote 6 procesado exitosamente.
Lote 7 procesado exitosamente.
Se han agregado 34886 documentos y generado sus embeddings.


Consulta aplicando ChromaDB

In [9]:
# Definir una consulta (puede ser una trama de película o cualquier texto)
query = "A young boy discovers "

# Realizar la consulta en la colección
results = collection.query(
    query_texts=[query],  # Pasar la consulta como texto
    n_results=5  # Número de resultados a devolver
)

# Mostrar los resultados
for i, document in enumerate(results['documents'][0]):  # Accedemos a la lista de documentos en el primer índice
    print(f"Resultado {i + 1}:")
    print(f"Trama: {document}")  # Solo mostramos la trama si no hay metadata disponible
    print("-" * 50)


Resultado 1:
Trama: An investigative thriller based on the search for a missing youngster.
--------------------------------------------------
Resultado 2:
Trama: A young boy from the lower caste resorts to petty thefts to make both ends meet after he lost his father at the early age. Once the boy realizes the importance of an education, he begins to improve his life and never looks back. Through diligence and dedication, he climbs the social and political ladder to success.
--------------------------------------------------
Resultado 3:
Trama: Thirteen-year-old Jesse is assigned a school project. A photographic self-portrait intended to portray one’s self without resorting to literal representation. Jesse lives with his parents, Sabi and Tim, in the lefty, middle class Toronto neighbourhood of Riverdale. A quiet and distant only-child with budding artistic aspirations, Jesse is inspired by the assignment to look for excitement and meaning in the world around him. Wielding a newly acqui